# Training Notebook for VQ-VAE

In [ ]:
# imports
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import VQ-VAE model from model.py
from model import Encoder, Decoder, Quantizer, EMBEDDING_DIM